[reference script](https://www.tensorflow.org/tutorials/structured_data/imbalanced_data)

# Import Package and Data



In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow import keras
import os
import tempfile
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler

Mounted at /content/drive


# Build the dataset


In [2]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Load your dataset from a CSV file
# Replace 'your_dataset.csv' with the path to your CSV file
# Make sure your CSV file contains features and a target column
# For example, X as features and y as the target column
# Modify this part according to your actual dataset structure
data = pd.read_csv("/content/drive/MyDrive/Wildfire/Datasets/merged_tp_precip_wind_fmc_all.csv")
data.dropna(inplace = True)
data.drop(["Unnamed: 0"], axis = 1, inplace = True)
data.drop(["START_DATE"], axis = 1, inplace = True)
data.drop(["END_DATE"], axis = 1, inplace = True)
#data.drop(["DATE"], axis = 1, inplace = True)
data["DATE"] = data["DATE"].astype(float)


In [21]:
data.groupby("FIRE_SIZE_CLASS").count()

,LATITUDE,LONGITUDE,FIPS,DATE,tmax,tmin,tavg,prcp,aws,fmc
FIRE_SIZE_CLASS,,,,,,,,,,
0,197739,197739,197739,197739,197739,197739,197739,197739,197739,197739
1,16518,16518,16518,16518,16518,16518,16518,16518,16518,16518
2,4063,4063,4063,4063,4063,4063,4063,4063,4063,4063


# Re-classification of the dataset with our own defined arces size

In [3]:
# Classify the ranges
def classify_ranges(df, column_name, ranges, classification_column):
    for range_name, range_values in ranges.items():
        df.loc[df[column_name].between(*range_values), classification_column] = range_name

# Redefine the classification
classification_ranges = {
    'A': (0, 99),
    'B': (99.000000000001, 999),
    'C': (999.000000000001, 99999999999999)
}
# Creating a new column to store the classifications
classify_ranges(data, 'FIRE_SIZE', classification_ranges, 'FIRE_SIZE_CLASS')
data.drop(["FIRE_SIZE"], axis = 1, inplace = True)
data.groupby("FIRE_SIZE_CLASS").count()
data['FIRE_SIZE_CLASS'] = data['FIRE_SIZE_CLASS'].replace({'A':0 , 'B':1 , 'C':2 ,'D':3 , 'E':4 , 'F':5 , 'G':6 })

# **Feedforward NN (afully connected neural network) for multi-class classification**

In [22]:
# Define your features and target column
X = data.drop('FIRE_SIZE_CLASS', axis=1)
y = data['FIRE_SIZE_CLASS']
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (optional but recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define your neural network model for 7-class classification
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')  # Use 'softmax' for 7-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Make predictions on the test set
y_pred = model.predict(X_test)

# Convert predicted probabilities to class labels
y_pred_labels = y_pred.argmax(axis=1)

# Print classification report
print(classification_report(y_test, y_pred_labels))

Epoch 1/10
5458/5458 [==============================] - 8s 1ms/step - loss: 0.3475 - accuracy: 0.9044 - val_loss: 0.3280 - val_accuracy: 0.9065
Epoch 2/10
5458/5458 [==============================] - 8s 2ms/step - loss: 0.3238 - accuracy: 0.9058 - val_loss: 0.3186 - val_accuracy: 0.9065
Epoch 3/10
5458/5458 [==============================] - 8s 1ms/step - loss: 0.3167 - accuracy: 0.9058 - val_loss: 0.3120 - val_accuracy: 0.9066
Epoch 4/10
5458/5458 [==============================] - 8s 1ms/step - loss: 0.3101 - accuracy: 0.9058 - val_loss: 0.3085 - val_accuracy: 0.9056
Epoch 5/10
5458/5458 [==============================] - 8s 2ms/step - loss: 0.3058 - accuracy: 0.9058 - val_loss: 0.3043 - val_accuracy: 0.9062
Epoch 6/10
5458/5458 [==============================] - 10s 2ms/step - loss: 0.3020 - accuracy: 0.9061 - val_loss: 0.3093 - val_accuracy: 0.9057
Epoch 7/10
5458/5458 [==============================] - 8s 1ms/step - loss: 0.2985 - accuracy: 0.9060 - val_loss: 0.2996 - val_accuracy

# **FNN**
Add more hidden layers and increase the number of neurons in each layer.

In [7]:
# Import necessary libraries
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Define your features and target column
X = data.drop('FIRE_SIZE_CLASS', axis=1)
y = data['FIRE_SIZE_CLASS']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (optional but recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define your neural network model for 7-class classification
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),  # Additional hidden layer
    tf.keras.layers.Dense(32, activation='relu'),  # Additional hidden layer
    tf.keras.layers.Dense(7, activation='softmax')  # Use 'softmax' for 7-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Make predictions on the test set
y_pred = model.predict(X_test)

# Convert predicted probabilities to class labels
y_pred_labels = y_pred.argmax(axis=1)

# Print classification report
print(classification_report(y_test, y_pred_labels))


Epoch 1/15
5458/5458 [==============================] - 15s 3ms/step - loss: 0.3376 - accuracy: 0.9054 - val_loss: 0.3184 - val_accuracy: 0.9065
Epoch 2/15
5458/5458 [==============================] - 14s 3ms/step - loss: 0.3129 - accuracy: 0.9059 - val_loss: 0.3079 - val_accuracy: 0.9065
Epoch 3/15
5458/5458 [==============================] - 15s 3ms/step - loss: 0.2976 - accuracy: 0.9060 - val_loss: 0.2931 - val_accuracy: 0.9074
Epoch 4/15
5458/5458 [==============================] - 14s 3ms/step - loss: 0.2840 - accuracy: 0.9070 - val_loss: 0.2835 - val_accuracy: 0.9080
Epoch 5/15
5458/5458 [==============================] - 14s 3ms/step - loss: 0.2746 - accuracy: 0.9078 - val_loss: 0.2703 - val_accuracy: 0.9088
Epoch 6/15
5458/5458 [==============================] - 14s 3ms/step - loss: 0.2675 - accuracy: 0.9087 - val_loss: 0.2664 - val_accuracy: 0.9097
Epoch 7/15
5458/5458 [==============================] - 13s 2ms/step - loss: 0.2617 - accuracy: 0.9096 - val_loss: 0.2589 - val_ac

# **FNN** - More recalls
To increase the recall of Class B and Class C, adjust the class weights during model training to give more importance to these minority classes. We should assign higher weights to the minority classes.

In [8]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import numpy as np

# Define your features and target column
X = data.drop('FIRE_SIZE_CLASS', axis=1)
y = data['FIRE_SIZE_CLASS']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (optional but recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Calculate class weights to give more importance to minority classes (B and C)
class_weights = {0: 1.0, 1: 5.0, 2: 5.0}  # Adjust the weights as needed

# Define your neural network model for 7-class classification
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),  # Additional hidden layer
    tf.keras.layers.Dense(32, activation='relu'),  # Additional hidden layer
    tf.keras.layers.Dense(7, activation='softmax')  # Use 'softmax' for 7-class classification
])

# Compile the model with class weights
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with class weights
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test), class_weight=class_weights)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Make predictions on the test set
y_pred = model.predict(X_test)

# Convert predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Print classification report
print(classification_report(y_test, y_pred_labels))


Epoch 1/15
5458/5458 [==============================] - 19s 3ms/step - loss: 1.0233 - accuracy: 0.8761 - val_loss: 0.4536 - val_accuracy: 0.8680
Epoch 2/15
5458/5458 [==============================] - 14s 3ms/step - loss: 0.9343 - accuracy: 0.8567 - val_loss: 0.4034 - val_accuracy: 0.8596
Epoch 3/15
5458/5458 [==============================] - 14s 3ms/step - loss: 0.8794 - accuracy: 0.8492 - val_loss: 0.4075 - val_accuracy: 0.8585
Epoch 4/15
5458/5458 [==============================] - 14s 3ms/step - loss: 0.8336 - accuracy: 0.8462 - val_loss: 0.4409 - val_accuracy: 0.8319
Epoch 5/15
5458/5458 [==============================] - 13s 2ms/step - loss: 0.8008 - accuracy: 0.8456 - val_loss: 0.4248 - val_accuracy: 0.8304
Epoch 6/15
5458/5458 [==============================] - 13s 2ms/step - loss: 0.7765 - accuracy: 0.8472 - val_loss: 0.4137 - val_accuracy: 0.8251
Epoch 7/15
5458/5458 [==============================] - 14s 3ms/step - loss: 0.7524 - accuracy: 0.8493 - val_loss: 0.4015 - val_ac

#FNN SMOTE
(Synthetic Minority Over-sampling Technique) for imbalanced dataset
pip install imbalanced-learn


In [9]:
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import numpy as np

# Assuming you have already loaded and prepared your data
# Define your features and target column
X = data.drop('FIRE_SIZE_CLASS', axis=1)
y = data['FIRE_SIZE_CLASS']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (optional but recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Apply SMOTE to balance the classes
sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train, y_train)

# Define your neural network model for 7-class classification
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_resampled.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')  # Use 'softmax' for 7-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model on the resampled data
model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Make predictions on the test set
y_pred = model.predict(X_test)

# Convert predicted probabilities to class labels
y_pred_labels = y_pred.argmax(axis=1)

# Print classification report
print(classification_report(y_test, y_pred_labels))


Epoch 1/10
14829/14829 [==============================] - 30s 2ms/step - loss: 0.8126 - accuracy: 0.6221 - val_loss: 0.8643 - val_accuracy: 0.6084
Epoch 2/10
14829/14829 [==============================] - 35s 2ms/step - loss: 0.6865 - accuracy: 0.6944 - val_loss: 0.6995 - val_accuracy: 0.7005
Epoch 3/10
14829/14829 [==============================] - 35s 2ms/step - loss: 0.6431 - accuracy: 0.7167 - val_loss: 0.7258 - val_accuracy: 0.6629
Epoch 4/10
14829/14829 [==============================] - 30s 2ms/step - loss: 0.6158 - accuracy: 0.7315 - val_loss: 0.7832 - val_accuracy: 0.6113
Epoch 5/10
14829/14829 [==============================] - 29s 2ms/step - loss: 0.5957 - accuracy: 0.7417 - val_loss: 0.6926 - val_accuracy: 0.6886
Epoch 6/10
14829/14829 [==============================] - 30s 2ms/step - loss: 0.5801 - accuracy: 0.7501 - val_loss: 0.6351 - val_accuracy: 0.7187
Epoch 7/10
14829/14829 [==============================] - 28s 2ms/step - loss: 0.5713 - accuracy: 0.7553 - val_loss: 0

# **RNN Model**
It provides more precisions.
RNNs work better for sequence data like time series or text data. They might not be the best choice for tabular data, as they are designed for sequential data. However, if your data has some sequential aspects, you can use a specific type of RNN called Long Short-Term Memory (LSTM) or Gated Recurrent Unit (GRU) for modeling sequences within the tabular data.

In [4]:
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Define your features and target column
X = data.drop('FIRE_SIZE_CLASS', axis=1)
y = data['FIRE_SIZE_CLASS']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (optional but recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define your neural network model with an LSTM layer
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')  # Use 'softmax' for 7-class classification
])

# Reshape the input data to match the expected input shape for LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Make predictions on the test set
y_pred = model.predict(X_test)

# Convert predicted probabilities to class labels
y_pred_labels = y_pred.argmax(axis=1)

# Print classification report
print(classification_report(y_test, y_pred_labels))


Epoch 1/10
5458/5458 [==============================] - 37s 6ms/step - loss: 0.3615 - accuracy: 0.9054 - val_loss: 0.3341 - val_accuracy: 0.9061
Epoch 2/10
5458/5458 [==============================] - 42s 8ms/step - loss: 0.3346 - accuracy: 0.9056 - val_loss: 0.3300 - val_accuracy: 0.9063
Epoch 3/10
5458/5458 [==============================] - 37s 7ms/step - loss: 0.3284 - accuracy: 0.9057 - val_loss: 0.3230 - val_accuracy: 0.9064
Epoch 4/10
5458/5458 [==============================] - 36s 7ms/step - loss: 0.3234 - accuracy: 0.9058 - val_loss: 0.3211 - val_accuracy: 0.9064
Epoch 5/10
5458/5458 [==============================] - 37s 7ms/step - loss: 0.3186 - accuracy: 0.9058 - val_loss: 0.3160 - val_accuracy: 0.9064
Epoch 6/10
5458/5458 [==============================] - 34s 6ms/step - loss: 0.3129 - accuracy: 0.9058 - val_loss: 0.3113 - val_accuracy: 0.9064
Epoch 7/10
5458/5458 [==============================] - 36s 7ms/step - loss: 0.3069 - accuracy: 0.9059 - val_loss: 0.3042 - val_ac

# **CNN**
CNNs are commonly used for image data, they can also be applied to tabular data when there are spatial relationships among features or if want to capture local patterns.

In [5]:
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Define your features and target column
X = data.drop('FIRE_SIZE_CLASS', axis=1)
y = data['FIRE_SIZE_CLASS']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (optional but recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the input data to match the expected input shape for CNN
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define your neural network model with a CNN layer
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')  # Use 'softmax' for 7-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Make predictions on the test set
y_pred = model.predict(X_test)

# Convert predicted probabilities to class labels
y_pred_labels = y_pred.argmax(axis=1)

# Print classification report
print(classification_report(y_test, y_pred_labels))


Epoch 1/10
5458/5458 [==============================] - 20s 4ms/step - loss: 0.3478 - accuracy: 0.9052 - val_loss: 0.3316 - val_accuracy: 0.9063
Epoch 2/10
5458/5458 [==============================] - 13s 2ms/step - loss: 0.3282 - accuracy: 0.9057 - val_loss: 0.3266 - val_accuracy: 0.9058
Epoch 3/10
5458/5458 [==============================] - 14s 3ms/step - loss: 0.3196 - accuracy: 0.9056 - val_loss: 0.3153 - val_accuracy: 0.9058
Epoch 4/10
5458/5458 [==============================] - 14s 3ms/step - loss: 0.3129 - accuracy: 0.9058 - val_loss: 0.3086 - val_accuracy: 0.9066
Epoch 5/10
5458/5458 [==============================] - 16s 3ms/step - loss: 0.3082 - accuracy: 0.9059 - val_loss: 0.3176 - val_accuracy: 0.9052
Epoch 6/10
5458/5458 [==============================] - 13s 2ms/step - loss: 0.3040 - accuracy: 0.9061 - val_loss: 0.3037 - val_accuracy: 0.9059
Epoch 7/10
5458/5458 [==============================] - 19s 3ms/step - loss: 0.3004 - accuracy: 0.9060 - val_loss: 0.3051 - val_ac

#CNN
Increase the number of layers in the CNN model to improve accuracy.
Added an additional Conv1D layer and dense layers to increase the depth of the model, which may help improve accuracy. We can further adjust the architecture and hyperparameters as needed to achieve the desired performance.

In [6]:
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Define your features and target column
X = data.drop('FIRE_SIZE_CLASS', axis=1)
y = data['FIRE_SIZE_CLASS']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (optional but recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the input data to match the expected input shape for CNN
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define your neural network model with multiple convolutional and dense layers
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')  # Use 'softmax' for 7-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Make predictions on the test set
y_pred = model.predict(X_test)

# Convert predicted probabilities to class labels
y_pred_labels = y_pred.argmax(axis=1)

# Print classification report
print(classification_report(y_test, y_pred_labels))


Epoch 1/10
5458/5458 [==============================] - 32s 5ms/step - loss: 0.3360 - accuracy: 0.9055 - val_loss: 0.3096 - val_accuracy: 0.9064
Epoch 2/10
5458/5458 [==============================] - 29s 5ms/step - loss: 0.3031 - accuracy: 0.9059 - val_loss: 0.2961 - val_accuracy: 0.9064
Epoch 3/10
5458/5458 [==============================] - 26s 5ms/step - loss: 0.2896 - accuracy: 0.9067 - val_loss: 0.2841 - val_accuracy: 0.9076
Epoch 4/10
5458/5458 [==============================] - 26s 5ms/step - loss: 0.2791 - accuracy: 0.9081 - val_loss: 0.2777 - val_accuracy: 0.9093
Epoch 5/10
5458/5458 [==============================] - 24s 4ms/step - loss: 0.2696 - accuracy: 0.9089 - val_loss: 0.2678 - val_accuracy: 0.9103
Epoch 6/10
5458/5458 [==============================] - 25s 5ms/step - loss: 0.2620 - accuracy: 0.9099 - val_loss: 0.2610 - val_accuracy: 0.9107
Epoch 7/10
5458/5458 [==============================] - 26s 5ms/step - loss: 0.2557 - accuracy: 0.9109 - val_loss: 0.2562 - val_ac